# Annotations

## Concatenation of rotations

In [2]:
#Use of numpy to handle matrices
import numpy as np

#Creation of the rotation matrices
Ry = np.matrix('0 0 -1; 0 1 0; 1 0 0')
Rz = np.matrix('-1 0 0; 0 -1 0; 0 0 1')
Rx = np.matrix('1 0 0; 1 1 0; 1 0 1')

print(np.matmul(Rz,np.matmul(Ry,Rx)))

[[ 1  0  1]
 [-1 -1  0]
 [ 1  0  0]]


The concatenation computed above is $R_z \cdot R_y \cdot R_x$, which can be interpreted in two different ways. First, if read from left to right, then the rotations are considered to be around **local axes**: $$(R_z \cdot R_{y'}) \cdot R_{x''}$$ That is, we first rotate around axis $z$, then we rotate around the **new axis** $y'$, and last we rotate around the **new axis** $x''$.

On the opposite, if read from right to left, the rotations are interpreted as rotations around **global axes**. That is, we first rotate around **global axis** $x$, then around **global axis** $y$, and last around **global axis** $z$: $$R_z \cdot (R_{y} \cdot R_{x})$$

## Euler Angles

The Euler angles matrix following the convention $zx'z''$  is computed as follows:

In [3]:
#We import sympy utilities to handle symbolic expressions in python
from sympy import symbols, Matrix, cos, sin

#Definition of the symbolic variables
alpha, beta, gamma = symbols('alpha beta gamma')

#Creation of rotation matrices
Rz = Matrix([[cos(alpha), -sin(alpha), 0],
             [sin(alpha), cos(alpha), 0],
             [0, 0, 1]])

Rx = Matrix([[1, 0, 0],
             [0, cos(beta), -sin(beta)],
             [0, sin(beta), cos(beta)]])

Rz2 = Matrix([[cos(gamma), -sin(gamma), 0],
              [sin(gamma), cos(gamma), 0],
              [0, 0, 1]])

#Internal 'Matrix' display method is used for a better visualization of the result
Concat = Rz*Rx*Rz2
Concat

Matrix([
[-sin(alpha)*sin(gamma)*cos(beta) + cos(alpha)*cos(gamma), -sin(alpha)*cos(beta)*cos(gamma) - sin(gamma)*cos(alpha),  sin(alpha)*sin(beta)],
[ sin(alpha)*cos(gamma) + sin(gamma)*cos(alpha)*cos(beta), -sin(alpha)*sin(gamma) + cos(alpha)*cos(beta)*cos(gamma), -sin(beta)*cos(alpha)],
[                                    sin(beta)*sin(gamma),                                     sin(beta)*cos(gamma),             cos(beta)]])

As follows are computed the Euler angles $(45°,30°,-45°)$ and $(0°,30°,-0°)$ with the convention $zx'z''$:

In [4]:
#Use of evalf Matrix method to evaluate the symbolic expressions
Concat.evalf(subs={alpha:np.pi/4, beta:np.pi/6, gamma:-np.pi/4})

Matrix([
[ 0.933012701892219, 0.0669872981077807,  0.353553390593274],
[0.0669872981077807,  0.933012701892219, -0.353553390593274],
[-0.353553390593274,  0.353553390593274,  0.866025403784439]])

In [5]:
Concat.evalf(subs={alpha:0, beta:np.pi/6, gamma:-0})

Matrix([
[1.0,                 0,                 0],
[  0, 0.866025403784439,              -0.5],
[  0,               0.5, 0.866025403784439]])

Now the Euler angles matrix for the convention $xyz$ (roll, pitch and yaw) is computed:

In [6]:
Rx = Matrix([[1,0,0],
             [0,cos(alpha),-sin(alpha)],
             [0,-sin(alpha),cos(alpha)]])

Ry = Matrix([[cos(beta),0,sin(beta)],
             [0,1,0],
             [-sin(beta),0,cos(beta)]])

Rz = Matrix([[cos(gamma),-sin(gamma),0],
             [sin(gamma),cos(gamma),0],
             [0,0,1]])

Rz*Ry*Rx

Matrix([
[cos(beta)*cos(gamma), -sin(alpha)*sin(beta)*cos(gamma) - sin(gamma)*cos(alpha),  sin(alpha)*sin(gamma) + sin(beta)*cos(alpha)*cos(gamma)],
[sin(gamma)*cos(beta), -sin(alpha)*sin(beta)*sin(gamma) + cos(alpha)*cos(gamma), -sin(alpha)*cos(gamma) + sin(beta)*sin(gamma)*cos(alpha)],
[          -sin(beta),                                    -sin(alpha)*cos(beta),                                     cos(alpha)*cos(beta)]])

# Exercise Sheet 1

## 1. Quaternions - Rotation matrices

Let $R_2$ be a rotation matrix, given by

$$R_2 = \begin{bmatrix}
0.36 & 0.48 & -0.8\\
-0.8 & 0.6 & 0\\
0.48 & 0.64 & 0.6
\end{bmatrix}$$

Calculate the quaternion $q$ that describes the rotation given by $R_2$

The vector describing the rotation axis $\overrightarrow{x}$ remains unchanged after the transformation is applied to it, therefore $\overrightarrow{x}$ is an **eigenvector** of the transformation $R_2$ with $\lambda=1$ $$R_2 \cdot \overrightarrow{x} = \overrightarrow{x}$$

Based on that, we have to solve the $3 \times 3$ system of equations that appears when computing $(R_2 - I)\overrightarrow{x}$ to find the vector $\overrightarrow{x}$

In [7]:
from sympy import solve, eye

x1, x2, x3 = symbols('x(1:4)')

R2 = np.matrix('0.36 0.48 -0.8; -0.8 0.6 0; 0.48 0.64 0.6')
x_v = Matrix([x1,x2,x3])

solve((R2-eye(3))*x_v)

{x1: -0.5*x3, x2: x3}

The result tells us that there are infinite solutions, however, to convert to quaternions is needed that the vector $\overrightarrow{x}$ has norm equal to 1, that means $||\overrightarrow{x}||=1$, which gives us another equation to solve and therefore a way to find an unique solution:

$$(\frac{-x_3^2}{2})^2 + 2x_3^2 = 1$$

In [8]:
solve([(-x3/2)**2 + 2*x3**2 - 1])

[{x3: -2/3}, {x3: 2/3}]

Then we choose our vector as

$$\overrightarrow{x} = \frac{1}{3}
\begin{bmatrix}
1\\
-2\\
-2
\end{bmatrix}$$

Now we find a vector $\overrightarrow{v}$, orthogonal to $\overrightarrow{x}$ and then we rotate it applying the rotation matrix to measure the angle

In [9]:
x = 1/3*np.matrix('1;-2;-2')
v1,v2,v3 = symbols('v(1:4)')

v = Matrix([v1,v2,v3])

solve(v.dot(x))

[{v1: 2.0*v2 + 2.0*v3}]

Therefore we choose

$$\overrightarrow{v} =
\begin{bmatrix}
2\\
\frac{1}{2}\\
\frac{1}{2}
\end{bmatrix}$$

and then we apply the transformation $R_2$ to this new vector in order to calculate the angle in between

In [10]:
v = np.matrix('2;0.5;0.5')
print(v.transpose().dot(x),end='\n\n')

v2 = R2*v
print(v2)

[[0.]]

[[ 0.56]
 [-1.3 ]
 [ 1.58]]


The resulting vector is

$$\overrightarrow{v'} = 
\begin{bmatrix}
0.56\\
-1.3\\
1.58
\end{bmatrix}
$$

The only thing remaining is to calculate the angle between $\overrightarrow{v}$ and $\overrightarrow{v'}$, which is done using the following:

$$\alpha = \arccos{\frac{\overrightarrow{v}\cdot\overrightarrow{v'}}{||\overrightarrow{v}||||\overrightarrow{v'}||}}$$

In [11]:
angle = np.arccos((v.transpose().dot(v2)/(np.linalg.norm(v)*np.linalg.norm(v2))).item(0))

print(f'angle = {angle*180/np.pi}')

angle = 73.73979529168804


The corresponding quaternion is then computed as follows:
$$q = (\cos(\frac{\alpha}{2}), \overrightarrow{x}\cdot\sin(\frac{\alpha}{2}))$$

In [12]:
q = np.matrix([np.cos(angle/2),x.item(0)*np.sin(angle/2),x.item(1)*np.sin(angle/2),x.item(2)*np.sin(angle/2)])
print(q)

[[ 0.8  0.2 -0.4 -0.4]]


## 2. Homogeneous matrices

Let $T \in SE(3)$ be a homogeneous transformation matrix and $\overrightarrow{v} = (1,2,3)^T$ be a vector, with

$$T = \begin{bmatrix}
0 & 0 & 1 & 5\\
0 & 1 & 0 & 0\\
-1 & 0 & 0 & 0\\
0 & 0 & 0 & 1
\end{bmatrix}$$

2) Apply the transformation described by $T$ to $\overrightarrow{v}$.

In [13]:
T = np.matrix('0 0 1 5;0 1 0 0; -1 0 0 0; 0 0 0 1')
v = np.matrix('1 2 3 1').transpose()
print(T*v)

[[ 8]
 [ 2]
 [-1]
 [ 1]]


3) Determine $T^{-1}$, being the inverse transformation matrix of $T$. 

In [14]:
#Use of inv function from package linalg
from numpy.linalg import inv

print(inv(T))

[[-0. -0. -1. -0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0. -5.]
 [ 0.  0.  0.  1.]]


## 3. Concatenation of Coordinate Transformations

We consider a service robot with a holonomic platform. The robot’s $x$ axis points in the direction of motion, and the $z$ axis points upwards. The $y$ axis is defined so that the coordinate system is right-handed. Let the initial pose of the robot in the basis coordinate system (BCS) be defined as

$$T_{init} = \begin{bmatrix}
1 & 0 & 0 & 5\\
0 & 1 & 0 & 3\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & 1
\end{bmatrix}$$

The following commands are consecutively sent to the service robot and executed:
1. Rotate around the z axis by 90◦.
2. Drive straight for 4 unit lengths.
3. Drive straight for 2 unit lengths, to the right for 3 unit lengths and finally rotate around the z axis by −45◦.

Calculate the transformation matrices corresponding to the individual commands, and the final pose of the robot in BCS.

To compute the solution we just have to understand how the concatenation of poses work. In this case the correct way to do it is:

$$T_{end} = T_{init} \cdot ^{init}T_1 \cdot ^1T_2 \cdot ^2T_3$$

In [15]:
alpha1, alpha2 = symbols('alpha(1:3)')

T_init = np.matrix('1 0 0 5; 0 1 0 3; 0 0 1 0; 0 0 0 1')

T_init_1 = Matrix([[cos(alpha1),-sin(alpha1),0,0],
                   [sin(alpha1),cos(alpha1),0,0],
                   [0,0,1,0],
                   [0,0,0,1]])

T_1_2 = Matrix([[1,0,0,4],
                [0,1,0,0],
                [0,0,1,0],
                [0,0,0,1]])

T_2_3 = Matrix([[cos(alpha2),-sin(alpha2),0,2],
                   [sin(alpha2),cos(alpha2),0,-3],
                   [0,0,1,0],
                   [0,0,0,1]])

display(T_init_1)
display(T_1_2)
display(T_2_3)

display((T_init*T_init_1*T_1_2*T_2_3).evalf(subs={alpha1:np.pi/2,alpha2:-np.pi/4}))

Matrix([
[cos(alpha1), -sin(alpha1), 0, 0],
[sin(alpha1),  cos(alpha1), 0, 0],
[          0,            0, 1, 0],
[          0,            0, 0, 1]])

Matrix([
[1, 0, 0, 4],
[0, 1, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]])

Matrix([
[cos(alpha2), -sin(alpha2), 0,  2],
[sin(alpha2),  cos(alpha2), 0, -3],
[          0,            0, 1,  0],
[          0,            0, 0,  1]])

Matrix([
[0.707106781186548, -0.707106781186547,   0, 8.0],
[0.707106781186547,  0.707106781186548,   0, 9.0],
[                0,                  0, 1.0,   0],
[                0,                  0,   0, 1.0]])

## 4. Distance Between Poses

In [16]:
Rtcp = np.matrix('0 -1 0; 1 0 0; 0 0 1')
Rgoal = np.matrix('0 0 -1; 0 1 0; 1 0 0')

print(np.matmul(Rgoal,Rtcp.transpose()),end='\n\n')
print(np.arccos(-1/2)*180/np.pi)

[[ 0  0 -1]
 [-1  0  0]
 [ 0  1  0]]

120.00000000000001
